In [1]:
%load_ext autoreload
%autoreload 2

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer

In [ ]:
df = pd.read_csv('Data/pf_suvs_i302_1s2025.csv')
df = df.drop(df.columns[0], axis=1)
mask = df['Moneda'] == '$'
df.loc[mask, 'Precio'] = df.loc[mask, 'Precio'] / 1185
df.drop(columns=['Moneda'], inplace=True)

df.drop(columns = ['Tipo de carrocería'],inplace=True)
idx = df[df['Año'] == 436694.0].index
df.drop(idx, inplace=True)

mappings = {
    'Con cámara de retroceso': {'No': 0, 'Sí': 1},
    'Transmisión': {
        'Automática': 0,
        'Manual': 1,
        'Automática secuencial': 2,
        'Semiautomática': 3
    }
}

for col, mapping in mappings.items():
    code_col = col.replace(' ', '_') + '_code'
    df[code_col] = df[col].map(mapping)

enc = OrdinalEncoder()
df[['Modelo_code', 'Version_code']] = enc.fit_transform(df[['Modelo', 'Versión']])

imputer = KNNImputer(n_neighbors=5, weights='uniform')
cols_to_impute = ['Modelo_code', 'Año', 'Version_code'] + [c.replace(' ', '_') + '_code' for c in mappings]
df[cols_to_impute] = imputer.fit_transform(df[cols_to_impute])

for col, mapping in mappings.items():
    code_col = col.replace(' ', '_') + '_code'
    df[code_col] = np.round(df[code_col]).astype(int)
    inv_map = {v: k for k, v in mapping.items()}
    df[col] = df[code_col].map(inv_map)

df['Puertas'] = df['Puertas'].replace(60252.0, np.nan)
imputer = KNNImputer(n_neighbors=5)
cols = ['Modelo_code', 'Año','Version_code','Puertas']
df[cols] = imputer.fit_transform(df[cols])
df['Puertas'] = np.round(df['Puertas']).astype(int)

aux_cols = ['Modelo_code', 'Version_code'] + [c.replace(' ', '_') + '_code' for c in mappings]
df.drop(aux_cols, axis=1, inplace=True)

print(df['Puertas'].value_counts(dropna=False))
print(df['Con cámara de retroceso'].value_counts(dropna=False))
print(df['Transmisión'].value_counts(dropna=False))

df['Kilómetros'] = (
    df['Kilómetros']
      .str.replace(r'\s*km$', '', regex=True)  
      .str.replace(r'\.', '', regex=True)      
      .astype(float)
)
print( df['Kilómetros'].value_counts(dropna=False))

# hechos transmision,con camara de retroceso,puertas,tipo de carroceria,moneda, precio,kilometros

Puertas
5    17734
4      399
3      105
2       10
6        3
7        2
Name: count, dtype: int64
Con cámara de retroceso
Sí    13778
No     4475
Name: count, dtype: int64
Transmisión
Automática               12292
Manual                    5837
Automática secuencial      123
Semiautomática               1
Name: count, dtype: int64


In [63]:
print(df.columns)
#Marca,Descripción,color
#Tipo de combustible,Motor,Tipo de vendedor,Versión,Título,Modelo
print( df['Color'].value_counts(dropna=False))





Index(['Marca', 'Modelo', 'Año', 'Versión', 'Color', 'Tipo de combustible',
       'Puertas', 'Transmisión', 'Motor', 'Con cámara de retroceso',
       'Kilómetros', 'Título', 'Precio', 'Descripción', 'Tipo de vendedor'],
      dtype='object')
Color
Gris                 5468
Blanco               4808
Negro                2825
Plateado             1427
Azul                 1121
                     ... 
MARRON KODIAK           1
blue                    1
BLACK MEET KETTLE       1
GRAY                    1
NOIR PERLA NERA         1
Name: count, Length: 71, dtype: int64


Kilómetros
0.0          4561
10000.0       220
1200000.0     194
900000.0      194
1000000.0     176
             ... 
962020.0        1
808000.0        1
1173760.0       1
918000.0        1
639000.0        1
Name: count, Length: 2146, dtype: int64
